In [35]:
import pandas as pd
from lib.mutation_util import date
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocessing (time filtering)

In [36]:
start_date = "2021-01-01"
end_date = "2021-11-30"

In [37]:
df_ori = pd.read_csv('../adyen-dataset.csv')
df_ori["date"] = df_ori.apply(date, axis=1)
df_ori["date"] = pd.to_datetime(df_ori["date"])
df_ori.set_index('date', inplace=True)
df_ts = df_ori.sort_index()
df_ts = df_ts.loc[start_date:end_date]
df_woe = df_ts.loc(axis=1)["psp_reference", "has_fraudulent_dispute", "ip_address", "email_address", "card_number"]
df_woe.reset_index(drop=True, inplace=True)
df_woe.head()

,psp_reference,has_fraudulent_dispute,ip_address,email_address,card_number
0,31441792706,False,qmHjXdZTQXznQZ2nK7S28A,6AIShI-nACdw5NTU_521kA,TU4KPVuB29KX7PZmtbhhUQ
1,56840734687,False,MoYL1fAsneKV6qc9TDsPnA,k93K-irSTKwd2pBkcXDK5w,71c2U4kmw9U_8vMaMd5YCQ
2,50841350701,True,yrHXrfW79VP4X1jyc7-eGw,LgvYsxCc7nbhLRujP0tYZQ,GvlEILP6lyz_nFf6bbDENg
3,81202637637,False,HtOkB2mBa5MG-ovenajWZg,RMJDQUiNhwzAnzCzOkUR-w,8X8Fb4r47wtQeghlxOuN0w
4,76149360397,False,nxVC8BgbuNyqXjZ_A8J2MQ,EGLNvgON4EdvfVMaHpn78Q,DdiFpiagUE6zrCRX6AcX0Q


In [41]:
from category_encoders import WOEEncoder

# woe encoding
enc1 = WOEEncoder()
ip_address_dummies = enc1.fit_transform(X=df_woe.ip_address, y=df_woe.has_fraudulent_dispute.values.tolist())
enc2 = WOEEncoder()
email_address_dummies = enc2.fit_transform(X=df_woe.email_address, y=df_woe.has_fraudulent_dispute.values.tolist())
enc3 = WOEEncoder()
card_number_dummies = enc3.fit_transform(X=df_woe.card_number, y=df_woe.has_fraudulent_dispute.values.tolist())

df_woe_features = pd.concat([df_woe["psp_reference"], df_woe["ip_address"], df_woe["email_address"], df_woe["card_number"], ip_address_dummies, email_address_dummies, card_number_dummies],axis=1)
symbols = ['psp_reference', 'ip_address','email_address','card_number','ip_address_woe', 'email_address_woe', 'card_number_woe']
df_woe_features.columns = symbols
df_woe_features

,psp_reference,ip_address,email_address,card_number,ip_address_woe,email_address_woe,card_number_woe
0,31441792706,qmHjXdZTQXznQZ2nK7S28A,6AIShI-nACdw5NTU_521kA,TU4KPVuB29KX7PZmtbhhUQ,0.000000,0.000000,0.000000
1,56840734687,MoYL1fAsneKV6qc9TDsPnA,k93K-irSTKwd2pBkcXDK5w,71c2U4kmw9U_8vMaMd5YCQ,0.000000,1.392115,1.392115
2,50841350701,yrHXrfW79VP4X1jyc7-eGw,LgvYsxCc7nbhLRujP0tYZQ,GvlEILP6lyz_nFf6bbDENg,2.308406,2.673049,2.308406
3,81202637637,HtOkB2mBa5MG-ovenajWZg,RMJDQUiNhwzAnzCzOkUR-w,8X8Fb4r47wtQeghlxOuN0w,0.000000,0.000000,0.000000
4,76149360397,nxVC8BgbuNyqXjZ_A8J2MQ,EGLNvgON4EdvfVMaHpn78Q,DdiFpiagUE6zrCRX6AcX0Q,1.392115,1.104433,1.104433
...,...,...,...,...,...,...,...
127045,61422715805,NaN,GlewPgEGPUohTbu7odbFPQ,0I_LNtaJId02RYsJYrdaMw,0.005309,0.000000,0.000000
127046,62714194430,ZC8fEYD5WgFNwn9yUrPSMA,ck1Qo_AYcLRtx6N41yl0mA,qxnnDmXl0DI-l8OuDElDvA,0.293503,0.000000,-0.148330
127047,77364072877,oukfYxaa8dqef3T-IfXrPA,XsQscJednbutwJ0koM3MoQ,EdjF0T_c6Isruja_8Yg0Wg,0.000000,1.392115,1.392115
127048,58757695028,lS18-mXp-7MN53HLG9k5rQ,RxTWOOXba45-iqcl6nZx2Q,3105RMBu7120IknwEVDAcA,0.000000,0.000000,0.000000


In [43]:
df_woe_features.to_csv(f"woe_features_{end_date}.csv", index=False)